In [ ]:
import pandas as pd
import plotly.express as px
import panel as pn
from itertools import product

import quantstats as qs
qs.extend_pandas()

import yfinance as yf
import datetime
import quantstats as qs

pn.extension("plotly")

import hvplot.pandas

def symbol_data(Symbols, years_back, freq = 'D'):
    end = datetime.datetime.now()
    start = datetime.datetime.now() - datetime.timedelta(days=365*years_back)
    # create empty dataframe
    stock_final = pd.DataFrame()
    # iterate over each symbol
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(Symbols,start=start, end=end, progress=False)

        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['Name']=Symbols
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None
        
    stock_final = stock_final.asfreq(freq, method='pad')
            
    return stock_final

def sharpe_ratio_calculator(tickers, years_back, freq):
    sharpe_list = []
    for ticker in tickers:
        ticker_dat = symbol_data(ticker, years_back, freq)
        ticker_returns = ticker_dat['Close'].pct_change()
        
        ticker_sharpe = ticker_returns.sharpe()
        sharpe_list.append(ticker_sharpe)
    return sharpe_list

def portfolio_calc_by_sharpe(tickers, years_back, freq):
    sharpe_list = sharpe_ratio_calculator(tickers, years_back, freq)
    
    sum_sharpe = sum(sharpe_list)
    ticker_proportion = {}
    for ticker, sharpe in zip(tickers, sharpe_list):
        print(f"{ticker} proportion = {round((sharpe/sum_sharpe) * 100,2)}%")
        ticker_proportion[ticker] = sharpe/sum_sharpe
    return ticker_proportion

def combiner_returns(tickers, years_back, freq):
    all_return_list = []
    for ticker in tickers:
        ticker_dat = symbol_data(ticker, years_back, freq)
        return_dat = ticker_dat['Close'].pct_change()
        
        all_return_list.append(return_dat)
    joined_return_dat = pd.concat(all_return_list, axis = 'columns', join = 'inner')
    return joined_return_dat

def mapping_sharpe_portfolio(tickers, years_back, freq):
    proportions = list(portfolio_calc_by_sharpe(tickers, years_back, freq).values())
    joined_return_dat = combiner_returns(tickers, years_back, freq)
    
    sharpe_portfolio_returns = joined_return_dat.dot(proportions)
    sharpe_portfolio_cumprod = (1 + sharpe_portfolio_returns).cumprod() - 1
    display(sharpe_portfolio_cumprod.hvplot(title = f"{tickers} portfolio"))
    


    
mapping_sharpe_portfolio(['AAPL', 'MSFT', 'SPY'], 3, 'W')   